You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [38]:
!pip install --upgrade crewai crewai_tools langchain_community
!pip install -U openai


Found existing installation: crewai 0.100.1
Uninstalling crewai-0.100.1:
  Successfully uninstalled crewai-0.100.1
Found existing installation: crewai-tools 0.33.0
Uninstalling crewai-tools-0.33.0:
  Successfully uninstalled crewai-tools-0.33.0
Found existing installation: langchain-community 0.3.17
Uninstalling langchain-community-0.3.17:
  Successfully uninstalled langchain-community-0.3.17
  Using cached crewai-0.100.1-py3-none-any.whl.metadata (27 kB)
  Using cached crewai_tools-0.33.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached langchain_community-0.3.17-py3-none-any.whl.metadata (2.4 kB)
Using cached crewai-0.100.1-py3-none-any.whl (234 kB)
Using cached crewai_tools-0.33.0-py3-none-any.whl (542 kB)
Using cached langchain_community-0.3.17-py3-none-any.whl (2.5 MB)


In [145]:
from langchain_openai import ChatOpenAI

In [66]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [82]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

- As a LLM for your agents, you'll be using OpenAI's `gpt-4o-mini`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os
from google.colab import userdata
openai_api_key = userdata.get("OPENAI_API_KEY")
grok_api_key = userdata.get("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [181]:
from crewai import Agent, Crew, Process, Task
from dotenv import load_dotenv
load_dotenv()

# Ensure your SerperDev API key is set in the environment variables
os.environ["SERPER_API_KEY"] = os.environ["SERPER_DEV_API_KEY"]

llm_ollama= ChatOpenAI(
    model_name="llama3.2:3b",
    base_url="http://localhost:11434/v1",
)

llm_openai = ChatOpenAI(
    model_name="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
)

llm_groq = ChatOpenAI(
    model_name="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY"),
)

llm  = llm_ollama

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [182]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic} using internet search",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."
              "You can search the nternet for latest information using the serper dev tool provdied to you.",
    allow_delegation=False,
    llm=llm,
    tools = [SerperDevTool()],
	verbose=True,
    max_retry_limit=3
)

### Agent: Writer

In [183]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic} using internet search",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "You can search the nternet for latest information using the serper dev tool.",
    allow_delegation=False,
    llm = llm,
    tools=[SerperDevTool()],
    verbose=True,
    max_retry_limit=5
)

### Agent: Editor

In [184]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm = llm,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [185]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    llm=llm,
)

### Task: Write

In [186]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    llm=llm,
)

### Task: Edit

In [187]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor,
    llm=llm,
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [188]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=0
)

2025-02-10 00:02:17,595 - 7950667776 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [189]:
result = crew.kickoff(inputs={"topic": "Elon musk is virtual president today."})



> Entering new CrewAgentExecutor chain...
Thought: To gather the latest trends, key players, and noteworthy news on Elon Musk's virtual presidency today, I need to search the internet for relevant information.
Action: Search the internet
Action Input: {“query": "Elon Musk virtual president today", “lang”: “en”}Thought:
To gather the latest trends, key players, and noteworthy news on Elon Musk's virtual presidency today, I need to search the internet for relevant information.

Action: Search the internet
Action Input: {“query": "Elon Musk virtual president today", “lang”: “en”}Thought: To gather the latest trends, key players, and noteworthy news on Elon Musk's virtual presidency today, I need to search the internet for relevant information.

Action: Search the internet
Action Input: {“query": "Elon Musk virtual president today", “lang”: “en”}Thought: To gather the latest trends, key players, and noteworthy news on Elon Musk's virtual presidency today, I need to search the internet fo

KeyboardInterrupt: 

- Display the results of your execution as markdown in the notebook.

In [179]:
from IPython.display import Markdown
Markdown(result)

**Elon Musk's Virtual Presidency: Understanding the Trends, Key Players, and Noteworthy News**
====================================================================

**Introduction**
---------------

Elon Musk, the CEO of SpaceX and Tesla, has been making headlines with his increasing influence in the Trump administration. As a special government employee, Musk has been wielding significant power, shaping policy and making decisions that affect the entire country. This raises important questions about the implications of his influence on the tech industry, the economy, and the future of politics. In this article, we'll explore the trends, key players, and noteworthy news surrounding Elon Musk's virtual presidency.

**Latest Trends and Key Players**
-------------------------------

Musk's role in the Trump administration has sparked controversy and debate. As a special government employee, he has been given unprecedented access to the White House, allowing him to shape policy and make decisions that affect the entire country. This has led to concerns about potential conflicts of interest, regulatory battles, and the impact of Musk's political ambitions on his business interests.

One of the key trends emerging from Musk's virtual presidency is the increasing blurring of lines between business and politics. With his dual role as CEO of SpaceX and Tesla, Musk has been able to leverage his influence to push for policies that benefit his companies. This has led to concerns about crony capitalism and the potential for abuse of power.

**Noteworthy News and Updates**
-----------------------------

Recent news and updates on Elon Musk's virtual presidency have focused on his cost-cutting efforts in government agencies. Musk has been tasked with identifying areas where the government can reduce spending, and his proposals have been met with both praise and criticism. While some have hailed his efforts as a necessary step towards fiscal responsibility, others have raised concerns about the potential impact on government services and the economy.

Another noteworthy development is Musk's use of social media to push his agenda. As the owner of X, Musk has been using the platform to promote his policies and influence public opinion. This has raised concerns about the potential for manipulation and the blurring of lines between politics and business.

**Conclusion**
--------------

Elon Musk's virtual presidency raises important questions about the intersection of business and politics. As we move forward, it's crucial that we pay attention to the trends, key players, and noteworthy news surrounding this phenomenon. By understanding the implications of Musk's influence, we can better navigate the complexities of modern politics and ensure that power is wielded responsibly.

In conclusion, Elon Musk's virtual presidency is a complex and multifaceted issue that requires careful consideration and analysis. As we continue to navigate the uncharted territory of tech mogul politics, it's essential that we remain vigilant and critical, ensuring that power is wielded in the best interests of the people, not just the powerful few.